# Multi-Agent Supply Chain Optimization

## Operations Research Use Case: End-to-End Supply Chain Planning

This notebook demonstrates a complex multi-agent system solving a supply chain optimization problem using:

- **4 Specialized Agents** working together
- **4 MCP Servers** with different tool sets
- **12+ Tools** for various operations
- **Workflow Visualization** showing agent collaboration

### Problem Statement

A company needs to optimize its supply chain operations:
1. **Forecast demand** for products across regions
2. **Optimize inventory** levels to minimize costs
3. **Plan delivery routes** to maximize efficiency
4. **Analyze costs** and provide strategic recommendations

### Architecture

```
Demand Forecaster → Inventory Optimizer → Route Planner → Cost Analyzer
     ↓                    ↓                    ↓              ↓
Analytics MCP      Optimization MCP      Logistics MCP   Finance MCP
```

In [1]:
# Setup: Add package to path (for development mode)
# import sys
# sys.path.insert(0, '/Users/aakashroy/Downloads/or-af framework development/or-af')

import or_af
from dotenv import load_dotenv
import json

load_dotenv()
print(f"OR-AF v{or_af.__version__} loaded successfully!")

OR-AF v0.4.0 loaded successfully!


## Configure Azure OpenAI Credentials

Set up your Azure OpenAI environment variables. Replace the placeholder values with your actual credentials.

In [2]:
import os

# Map your .env variables to what or-af expects
os.environ["subscription_key"] = os.getenv("AZURE_OPENAI_API_KEY")
os.environ["api_version"] = os.getenv("AZURE_OPENAI_API_VERSION")
os.environ["endpoint"] = os.getenv("AZURE_OPENAI_ENDPOINT")
os.environ["deployment"] = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

print("✓ Environment variable mapping configured")
print(f"  subscription_key: {'✓ Set' if os.getenv('subscription_key') else '✗ Not set'}")
print(f"  api_version: {'✓ Set' if os.getenv('api_version') else '✗ Not set'}")
print(f"  endpoint: {'✓ Set' if os.getenv('endpoint') else '✗ Not set'}")
print(f"  deployment: {'✓ Set' if os.getenv('deployment') else '✗ Not set'}")

✓ Environment variable mapping configured
  subscription_key: ✓ Set
  api_version: ✓ Set
  endpoint: ✓ Set
  deployment: ✓ Set


## Step 1: Create MCP Servers with Specialized Tools

We'll create four MCP servers, each hosting domain-specific tools.

### 1.1 Analytics MCP Server

Tools for statistical analysis and demand forecasting.

In [3]:
from or_af import create_mcp_server
import random
from typing import List, Dict, Any

# Create Analytics MCP Server
analytics_server = create_mcp_server(
    name="analytics_mcp",
    description="Statistical analysis and forecasting tools"
)

@analytics_server.tool()
def calculate_moving_average(data: List[float], window: int = 3) -> float:
    """Calculate moving average for demand forecasting.
    
    Args:
        data: Historical demand data
        window: Window size for moving average
    
    Returns:
        Moving average value
    """
    if len(data) < window:
        return sum(data) / len(data)
    return sum(data[-window:]) / window

@analytics_server.tool()
def forecast_demand(historical_data: List[float], periods: int = 1) -> List[float]:
    """Forecast future demand using exponential smoothing.
    
    Args:
        historical_data: Past demand values
        periods: Number of periods to forecast
    
    Returns:
        List of forecasted demand values
    """
    alpha = 0.3  # Smoothing factor
    forecast = [historical_data[-1]]  # Start with last known value
    
    for _ in range(periods):
        # Simple exponential smoothing with trend
        trend = sum(historical_data[-3:]) / 3 - sum(historical_data[-6:-3]) / 3
        next_val = forecast[-1] * (1 + alpha) + trend * 0.5
        forecast.append(max(0, next_val))  # Ensure non-negative
    
    return forecast[1:]  # Return only new forecasts

@analytics_server.tool()
def calculate_statistics(data: List[float]) -> Dict[str, float]:
    """Calculate statistical measures for demand data.
    
    Args:
        data: Numerical data to analyze
    
    Returns:
        Dictionary with mean, std, min, max
    """
    import statistics
    return {
        "mean": statistics.mean(data),
        "std_dev": statistics.stdev(data) if len(data) > 1 else 0,
        "min": min(data),
        "max": max(data)
    }

print(f"✓ Analytics MCP Server created")
print(f"  Tools: {analytics_server.list_tools()}")

00:33:29 | INFO | MCP Server 'analytics_mcp' initialized (using official MCP SDK)


[02/01/26 00:33:29] INFO     MCP Server 'analytics_mcp' initialized (using official MCP SDK)           ]8;id=46589;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py\logger.py]8;;\:]8;id=974075;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py#65\65]8;;\

00:33:29 | INFO | Tool 'calculate_moving_average' registered with MCP server 'analytics_mcp'


                    INFO     Tool 'calculate_moving_average' registered with MCP server                ]8;id=573179;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py\logger.py]8;;\:]8;id=577640;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py#65\65]8;;\
                             'analytics_mcp'                                                                       

00:33:29 | INFO | Tool 'forecast_demand' registered with MCP server 'analytics_mcp'


                    INFO     Tool 'forecast_demand' registered with MCP server 'analytics_mcp'         ]8;id=180010;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py\logger.py]8;;\:]8;id=866377;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py#65\65]8;;\

00:33:29 | INFO | Tool 'calculate_statistics' registered with MCP server 'analytics_mcp'


                    INFO     Tool 'calculate_statistics' registered with MCP server 'analytics_mcp'    ]8;id=529285;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py\logger.py]8;;\:]8;id=544288;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py#65\65]8;;\

✓ Analytics MCP Server created
  Tools: ['calculate_moving_average', 'forecast_demand', 'calculate_statistics']


### 1.2 Optimization MCP Server

Tools for inventory optimization and constraint solving.

In [4]:
# Create Optimization MCP Server
optimization_server = create_mcp_server(
    name="optimization_mcp",
    description="Inventory and resource optimization tools"
)

@optimization_server.tool()
def calculate_eoq(annual_demand: float, ordering_cost: float, holding_cost: float) -> Dict[str, float]:
    """Calculate Economic Order Quantity (EOQ) - classic OR model.
    
    Args:
        annual_demand: Total annual demand
        ordering_cost: Cost per order
        holding_cost: Annual holding cost per unit
    
    Returns:
        Dictionary with EOQ, total cost, and order frequency
    """
    import math
    
    eoq = math.sqrt((2 * annual_demand * ordering_cost) / holding_cost)
    num_orders = annual_demand / eoq
    total_cost = (annual_demand / eoq) * ordering_cost + (eoq / 2) * holding_cost
    
    return {
        "optimal_order_quantity": round(eoq, 2),
        "total_annual_cost": round(total_cost, 2),
        "orders_per_year": round(num_orders, 2),
        "days_between_orders": round(365 / num_orders, 2)
    }

@optimization_server.tool()
def calculate_safety_stock(avg_demand: float, std_demand: float, lead_time: int, service_level: float = 0.95) -> Dict[str, float]:
    """Calculate safety stock based on demand variability.
    
    Args:
        avg_demand: Average daily demand
        std_demand: Standard deviation of demand
        lead_time: Lead time in days
        service_level: Desired service level (0-1)
    
    Returns:
        Dictionary with safety stock and reorder point
    """
    import math
    
    # Z-score for service level (approximation)
    z_scores = {0.90: 1.28, 0.95: 1.65, 0.99: 2.33}
    z = z_scores.get(service_level, 1.65)
    
    safety_stock = z * std_demand * math.sqrt(lead_time)
    reorder_point = avg_demand * lead_time + safety_stock
    
    return {
        "safety_stock": round(safety_stock, 2),
        "reorder_point": round(reorder_point, 2),
        "service_level": service_level
    }

@optimization_server.tool()
def optimize_inventory_levels(demand_forecast: List[float], current_stock: float, target_service_level: float = 0.95) -> Dict[str, float]:
    """Determine optimal inventory levels based on forecast.
    
    Args:
        demand_forecast: Forecasted demand values
        current_stock: Current inventory level
        target_service_level: Target service level
    
    Returns:
        Recommended inventory actions
    """
    import statistics
    
    avg_forecast = statistics.mean(demand_forecast)
    std_forecast = statistics.stdev(demand_forecast) if len(demand_forecast) > 1 else avg_forecast * 0.2
    
    # Calculate required stock
    z = 1.65 if target_service_level == 0.95 else 1.28
    required_stock = sum(demand_forecast) + z * std_forecast * len(demand_forecast) ** 0.5
    
    return {
        "current_stock": current_stock,
        "required_stock": round(required_stock, 2),
        "order_quantity": round(max(0, required_stock - current_stock), 2),
        "stock_coverage_days": round(current_stock / avg_forecast if avg_forecast > 0 else 0, 2)
    }

print(f"✓ Optimization MCP Server created")
print(f"  Tools: {optimization_server.list_tools()}")

00:33:29 | INFO | MCP Server 'optimization_mcp' initialized (using official MCP SDK)


                    INFO     MCP Server 'optimization_mcp' initialized (using official MCP SDK)        ]8;id=550803;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py\logger.py]8;;\:]8;id=374971;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py#65\65]8;;\

00:33:29 | INFO | Tool 'calculate_eoq' registered with MCP server 'optimization_mcp'


                    INFO     Tool 'calculate_eoq' registered with MCP server 'optimization_mcp'        ]8;id=46679;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py\logger.py]8;;\:]8;id=267985;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py#65\65]8;;\

00:33:29 | INFO | Tool 'calculate_safety_stock' registered with MCP server 'optimization_mcp'


                    INFO     Tool 'calculate_safety_stock' registered with MCP server                  ]8;id=738634;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py\logger.py]8;;\:]8;id=843101;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py#65\65]8;;\
                             'optimization_mcp'                                                                    

00:33:29 | INFO | Tool 'optimize_inventory_levels' registered with MCP server 'optimization_mcp'


                    INFO     Tool 'optimize_inventory_levels' registered with MCP server               ]8;id=775626;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py\logger.py]8;;\:]8;id=342382;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py#65\65]8;;\
                             'optimization_mcp'                                                                    

✓ Optimization MCP Server created
  Tools: ['calculate_eoq', 'calculate_safety_stock', 'optimize_inventory_levels']


### 1.3 Logistics MCP Server

Tools for route planning and delivery optimization.

In [5]:
# Create Logistics MCP Server
logistics_server = create_mcp_server(
    name="logistics_mcp",
    description="Route planning and delivery optimization tools"
)

@logistics_server.tool()
def calculate_distance(location1: str, location2: str) -> float:
    """Calculate distance between two locations (simplified).
    
    Args:
        location1: Starting location
        location2: Ending location
    
    Returns:
        Distance in km
    """
    # Simplified distance calculation (in real scenario, use actual coordinates)
    distances = {
        ("warehouse", "region_A"): 150,
        ("warehouse", "region_B"): 200,
        ("warehouse", "region_C"): 120,
        ("region_A", "region_B"): 180,
        ("region_B", "region_C"): 160,
        ("region_A", "region_C"): 140
    }
    
    key = (location1, location2)
    reverse_key = (location2, location1)
    
    return distances.get(key, distances.get(reverse_key, 100))

@logistics_server.tool()
def plan_route(start: str, destinations: List[str]) -> Dict[str, Any]:
    """Plan optimal delivery route using nearest neighbor heuristic.
    
    Args:
        start: Starting location
        destinations: List of delivery locations
    
    Returns:
        Optimal route and total distance
    """
    route = [start]
    remaining = destinations.copy()
    current = start
    total_distance = 0
    
    # Nearest neighbor algorithm
    while remaining:
        nearest = min(remaining, key=lambda x: calculate_distance(current, x))
        distance = calculate_distance(current, nearest)
        route.append(nearest)
        total_distance += distance
        current = nearest
        remaining.remove(nearest)
    
    # Return to start
    total_distance += calculate_distance(current, start)
    route.append(start)
    
    return {
        "route": route,
        "total_distance_km": round(total_distance, 2),
        "num_stops": len(destinations)
    }

@logistics_server.tool()
def calculate_vehicle_capacity(total_demand: float, vehicle_capacity: float = 1000) -> Dict[str, int]:
    """Calculate number of vehicles needed for delivery.
    
    Args:
        total_demand: Total units to deliver
        vehicle_capacity: Capacity per vehicle
    
    Returns:
        Vehicle requirements
    """
    import math
    
    vehicles_needed = math.ceil(total_demand / vehicle_capacity)
    
    return {
        "vehicles_needed": vehicles_needed,
        "total_capacity": vehicles_needed * vehicle_capacity,
        "utilization_percent": round((total_demand / (vehicles_needed * vehicle_capacity)) * 100, 2)
    }

print(f"✓ Logistics MCP Server created")
print(f"  Tools: {logistics_server.list_tools()}")

00:33:29 | INFO | MCP Server 'logistics_mcp' initialized (using official MCP SDK)


                    INFO     MCP Server 'logistics_mcp' initialized (using official MCP SDK)           ]8;id=756553;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py\logger.py]8;;\:]8;id=490884;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py#65\65]8;;\

00:33:29 | INFO | Tool 'calculate_distance' registered with MCP server 'logistics_mcp'


                    INFO     Tool 'calculate_distance' registered with MCP server 'logistics_mcp'      ]8;id=747021;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py\logger.py]8;;\:]8;id=614396;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py#65\65]8;;\

00:33:29 | INFO | Tool 'plan_route' registered with MCP server 'logistics_mcp'


                    INFO     Tool 'plan_route' registered with MCP server 'logistics_mcp'              ]8;id=787719;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py\logger.py]8;;\:]8;id=310950;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py#65\65]8;;\

00:33:29 | INFO | Tool 'calculate_vehicle_capacity' registered with MCP server 'logistics_mcp'


                    INFO     Tool 'calculate_vehicle_capacity' registered with MCP server              ]8;id=291963;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py\logger.py]8;;\:]8;id=871099;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py#65\65]8;;\
                             'logistics_mcp'                                                                       

✓ Logistics MCP Server created
  Tools: ['calculate_distance', 'plan_route', 'calculate_vehicle_capacity']


### 1.4 Finance MCP Server

Tools for cost analysis and financial optimization.

In [6]:
# Create Finance MCP Server
finance_server = create_mcp_server(
    name="finance_mcp",
    description="Cost analysis and financial optimization tools"
)

@finance_server.tool()
def calculate_inventory_costs(avg_inventory: float, holding_cost_per_unit: float, order_quantity: float, ordering_cost: float, num_orders: float) -> Dict[str, float]:
    """Calculate total inventory-related costs.
    
    Args:
        avg_inventory: Average inventory level
        holding_cost_per_unit: Annual holding cost per unit
        order_quantity: Quantity per order
        ordering_cost: Cost per order
        num_orders: Number of orders per year
    
    Returns:
        Breakdown of inventory costs
    """
    holding_cost = avg_inventory * holding_cost_per_unit
    ordering_cost_total = num_orders * ordering_cost
    total_cost = holding_cost + ordering_cost_total
    
    return {
        "holding_cost": round(holding_cost, 2),
        "ordering_cost": round(ordering_cost_total, 2),
        "total_inventory_cost": round(total_cost, 2)
    }

@finance_server.tool()
def calculate_transportation_costs(distance_km: float, cost_per_km: float, num_vehicles: int, fixed_cost_per_vehicle: float = 500) -> Dict[str, float]:
    """Calculate transportation costs.
    
    Args:
        distance_km: Total distance traveled
        cost_per_km: Variable cost per kilometer
        num_vehicles: Number of vehicles used
        fixed_cost_per_vehicle: Fixed cost per vehicle
    
    Returns:
        Transportation cost breakdown
    """
    variable_cost = distance_km * cost_per_km
    fixed_cost = num_vehicles * fixed_cost_per_vehicle
    total_cost = variable_cost + fixed_cost
    
    return {
        "variable_cost": round(variable_cost, 2),
        "fixed_cost": round(fixed_cost, 2),
        "total_transportation_cost": round(total_cost, 2),
        "cost_per_km": cost_per_km
    }

@finance_server.tool()
def calculate_total_supply_chain_cost(inventory_cost: float, transportation_cost: float, stockout_cost: float = 0) -> Dict[str, float]:
    """Calculate total supply chain cost and provide insights.
    
    Args:
        inventory_cost: Total inventory costs
        transportation_cost: Total transportation costs
        stockout_cost: Cost of stockouts/lost sales
    
    Returns:
        Total cost and cost breakdown percentages
    """
    total_cost = inventory_cost + transportation_cost + stockout_cost
    
    return {
        "total_supply_chain_cost": round(total_cost, 2),
        "inventory_cost_percent": round((inventory_cost / total_cost) * 100, 2) if total_cost > 0 else 0,
        "transportation_cost_percent": round((transportation_cost / total_cost) * 100, 2) if total_cost > 0 else 0,
        "stockout_cost_percent": round((stockout_cost / total_cost) * 100, 2) if total_cost > 0 else 0
    }

print(f"✓ Finance MCP Server created")
print(f"  Tools: {finance_server.list_tools()}")

00:33:29 | INFO | MCP Server 'finance_mcp' initialized (using official MCP SDK)


                    INFO     MCP Server 'finance_mcp' initialized (using official MCP SDK)             ]8;id=718439;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py\logger.py]8;;\:]8;id=391449;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py#65\65]8;;\

00:33:29 | INFO | Tool 'calculate_inventory_costs' registered with MCP server 'finance_mcp'


                    INFO     Tool 'calculate_inventory_costs' registered with MCP server 'finance_mcp' ]8;id=112445;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py\logger.py]8;;\:]8;id=125651;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py#65\65]8;;\

00:33:29 | INFO | Tool 'calculate_transportation_costs' registered with MCP server 'finance_mcp'


                    INFO     Tool 'calculate_transportation_costs' registered with MCP server          ]8;id=396536;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py\logger.py]8;;\:]8;id=488022;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py#65\65]8;;\
                             'finance_mcp'                                                                         

00:33:29 | INFO | Tool 'calculate_total_supply_chain_cost' registered with MCP server 'finance_mcp'


                    INFO     Tool 'calculate_total_supply_chain_cost' registered with MCP server       ]8;id=162815;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py\logger.py]8;;\:]8;id=955655;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py#65\65]8;;\
                             'finance_mcp'                                                                         

✓ Finance MCP Server created
  Tools: ['calculate_inventory_costs', 'calculate_transportation_costs', 'calculate_total_supply_chain_cost']


## Step 2: Create Specialized Agents

Each agent has access to its domain-specific MCP server and is optimized for its role.

In [7]:
from or_af import Agent

# Agent 1: Demand Forecaster
demand_forecaster = Agent(
    name="demand_forecaster",
    system_prompt="""You are a Demand Forecasting specialist in supply chain optimization.
    
    Your role:
    - Analyze historical demand data
    - Calculate statistical measures
    - Forecast future demand using available tools
    - Provide insights on demand patterns and trends
    
    Always use the analytics tools to perform calculations and provide data-driven forecasts.
    Format your output as JSON with keys: forecast, statistics, insights.""",
    mcp_servers=[analytics_server],
    max_iterations=5
)

# Agent 2: Inventory Optimizer
inventory_optimizer = Agent(
    name="inventory_optimizer",
    system_prompt="""You are an Inventory Optimization expert specializing in Operations Research.
    
    Your role:
    - Calculate Economic Order Quantity (EOQ)
    - Determine safety stock levels
    - Optimize inventory levels based on demand forecasts
    - Recommend reorder points and order quantities
    
    Use optimization tools to calculate optimal inventory parameters.
    Format your output as JSON with keys: eoq_analysis, safety_stock, inventory_recommendations.""",
    mcp_servers=[optimization_server],
    max_iterations=5
)

# Agent 3: Route Planner
route_planner = Agent(
    name="route_planner",
    system_prompt="""You are a Logistics and Route Planning specialist.
    
    Your role:
    - Plan optimal delivery routes
    - Calculate distances and travel times
    - Determine vehicle requirements
    - Optimize delivery efficiency
    
    Use logistics tools to create efficient delivery plans.
    Format your output as JSON with keys: route_plan, vehicle_requirements, logistics_summary.""",
    mcp_servers=[logistics_server],
    max_iterations=5
)

# Agent 4: Cost Analyzer
cost_analyzer = Agent(
    name="cost_analyzer",
    system_prompt="""You are a Supply Chain Cost Analysis and Financial Optimization expert.
    
    Your role:
    - Calculate inventory costs
    - Analyze transportation costs
    - Compute total supply chain costs
    - Provide cost reduction recommendations
    - Generate executive summary with strategic insights
    
    Use finance tools to perform comprehensive cost analysis.
    Format your final output as JSON with keys: cost_breakdown, total_cost, recommendations, executive_summary.""",
    mcp_servers=[finance_server],
    max_iterations=5
)

print("✓ All 4 specialized agents created:")
print(f"  1. {demand_forecaster.name} -> {demand_forecaster.list_available_tools()}")
print(f"  2. {inventory_optimizer.name} -> {inventory_optimizer.list_available_tools()}")
print(f"  3. {route_planner.name} -> {route_planner.list_available_tools()}")
print(f"  4. {cost_analyzer.name} -> {cost_analyzer.list_available_tools()}")

00:33:29 | INFO | Agent 'demand_forecaster' connected to MCP server 'analytics_mcp'


                    INFO     Agent 'demand_forecaster' connected to MCP server 'analytics_mcp'         ]8;id=154405;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py\logger.py]8;;\:]8;id=928775;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py#65\65]8;;\

✓ Connected to MCP server 'analytics_mcp' (3 tools)
00:33:29 | INFO | Agent 'demand_forecaster' initialized with model: gpt-5


                    INFO     Agent 'demand_forecaster' initialized with model: gpt-5                   ]8;id=32333;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py\logger.py]8;;\:]8;id=674909;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py#65\65]8;;\

00:33:29 | INFO | Agent 'inventory_optimizer' connected to MCP server 'optimization_mcp'


...

00:33:29 | INFO | Agent 'cost_analyzer' connected to MCP server 'finance_mcp'


                    INFO     Agent 'cost_analyzer' connected to MCP server 'finance_mcp'               ]8;id=969106;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py\logger.py]8;;\:]8;id=470340;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py#65\65]8;;\

✓ Connected to MCP server 'finance_mcp' (3 tools)
00:33:29 | INFO | Agent 'cost_analyzer' initialized with model: gpt-5


                    INFO     Agent 'cost_analyzer' initialized with model: gpt-5                       ]8;id=499577;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py\logger.py]8;;\:]8;id=702519;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py#65\65]8;;\

✓ All 4 specialized agents created:
  1. demand_forecaster -> {'analytics_mcp': ['calculate_moving_average', 'forecast_demand', 'calculate_statistics']}
  2. inventory_optimizer -> {'optimization_mcp': ['calculate_eoq', 'calculate_safety_stock', 'optimize_inventory_levels']}
  3. route_planner -> {'logistics_mcp': ['calculate_distance', 'plan_route', 'calculate_vehicle_capacity']}
  4. cost_analyzer -> {'finance_mcp': ['calculate_inventory_costs', 'calculate_transportation_costs', 'calculate_total_supply_chain_cost']}


## Step 3: Build Multi-Agent Workflow

Create a workflow where agents collaborate sequentially, with each agent's output feeding into the next.

In [8]:
from or_af import WorkflowGraph, EdgeCondition

# Create the workflow graph
supply_chain_workflow = WorkflowGraph(
    name="supply_chain_optimization",
    description="End-to-end supply chain optimization workflow"
)

# Add agent nodes to the workflow
forecast_node = supply_chain_workflow.add_node(
    demand_forecaster,
    name="Demand_Forecasting",
    description="Forecast future demand",
    is_entry=True
)

inventory_node = supply_chain_workflow.add_node(
    inventory_optimizer,
    name="Inventory_Optimization",
    description="Optimize inventory levels"
)

route_node = supply_chain_workflow.add_node(
    route_planner,
    name="Route_Planning",
    description="Plan delivery routes"
)

cost_node = supply_chain_workflow.add_node(
    cost_analyzer,
    name="Cost_Analysis",
    description="Analyze costs and provide recommendations",
    is_exit=True
)

# Connect nodes with edges
supply_chain_workflow.add_edge(forecast_node, inventory_node, condition=EdgeCondition.ON_SUCCESS)
supply_chain_workflow.add_edge(inventory_node, route_node, condition=EdgeCondition.ON_SUCCESS)
supply_chain_workflow.add_edge(route_node, cost_node, condition=EdgeCondition.ON_SUCCESS)

# Compile the workflow
supply_chain_workflow.compile()

print("✓ Multi-Agent Workflow Created!")
print(f"  Workflow: {supply_chain_workflow.name}")
print(f"  Nodes: {len(supply_chain_workflow.nodes)}")
print(f"  Edges: {len(supply_chain_workflow.edges)}")
print(f"  Entry: {supply_chain_workflow.entry_node.name}")
print(f"  Exit: {[supply_chain_workflow.nodes[n].name for n in supply_chain_workflow.exit_nodes]}")

00:33:29 | INFO | Node 'Demand_Forecasting' added to workflow 'supply_chain_optimization'


                    INFO     Node 'Demand_Forecasting' added to workflow 'supply_chain_optimization'   ]8;id=991432;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py\logger.py]8;;\:]8;id=139798;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py#65\65]8;;\

00:33:29 | INFO | Node 'Inventory_Optimization' added to workflow 'supply_chain_optimization'


                    INFO     Node 'Inventory_Optimization' added to workflow                           ]8;id=697220;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py\logger.py]8;;\:]8;id=216313;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py#65\65]8;;\
                             'supply_chain_optimization'                                                           

00:33:29 | INFO | Node 'Route_Planning' added to workflow 'supply_chain_optimization'


...

00:33:29 | INFO | Edge 'Route_Planning->Cost_Analysis' added to workflow


                    INFO     Edge 'Route_Planning->Cost_Analysis' added to workflow                    ]8;id=381382;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py\logger.py]8;;\:]8;id=748423;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py#65\65]8;;\

00:33:29 | INFO | Workflow 'supply_chain_optimization' compiled successfully


                    INFO     Workflow 'supply_chain_optimization' compiled successfully                ]8;id=987697;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py\logger.py]8;;\:]8;id=588560;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py#65\65]8;;\

✓ Multi-Agent Workflow Created!
  Workflow: supply_chain_optimization
  Nodes: 4
  Edges: 3
  Entry: Demand_Forecasting
  Exit: ['Cost_Analysis']


## Step 4: Visualize the Workflow

**Key Feature**: OR-AF provides multiple visualization formats to understand complex workflows.

In [9]:
print("=" * 70)
print("WORKFLOW VISUALIZATION - TEXT FORMAT")
print("=" * 70)
print(supply_chain_workflow.visualize(format="text"))

print("\n" + "=" * 70)
print("WORKFLOW VISUALIZATION - ASCII GRAPH")
print("=" * 70)
print(supply_chain_workflow.visualize(format="ascii"))

print("\n" + "=" * 70)
print("WORKFLOW VISUALIZATION - MERMAID DIAGRAM")
print("=" * 70)
print(supply_chain_workflow.visualize(format="mermaid"))

WORKFLOW VISUALIZATION - TEXT FORMAT
📊 Workflow: supply_chain_optimization
📝 Description: End-to-end supply chain optimization workflow
🔢 Nodes: 4 | Edges: 3

🔷 NODES:
----------------------------------------
  ⏳ Demand_Forecasting 🚀[ENTRY]
      📡 MCP Servers:
          └─ analytics_mcp
  ⏳ Inventory_Optimization
      📡 MCP Servers:
          └─ optimization_mcp
  ⏳ Route_Planning
      📡 MCP Servers:
          └─ logistics_mcp
  ⏳ Cost_Analysis 🏁[EXIT]
      📡 MCP Servers:
          └─ finance_mcp

🔗 EDGES:
----------------------------------------
  Demand_Forecasting ✅──▶ Inventory_Optimization
      Condition: on_success
  Inventory_Optimization ✅──▶ Route_Planning
      Condition: on_success
  Route_Planning ✅──▶ Cost_Analysis
      Condition: on_success

WORKFLOW VISUALIZATION - ASCII GRAPH
+==========================================================+
|                supply_chain_optimization                 |
+==========================================================+

+------

## Step 5: Define the Supply Chain Problem

Set up a realistic supply chain scenario with historical data.

In [10]:
# Supply Chain Problem Parameters
problem_data = {
    "company": "Global Manufacturing Co.",
    "product": "Industrial Components",
    "regions": ["region_A", "region_B"],
    
    # Historical demand data (last 6 months)
    "historical_demand": [850, 920, 950, 980, 1050, 1100],
    
    # Current inventory
    "current_inventory": 1200,
    
    # Cost parameters
    "ordering_cost": 500,  # $ per order
    "holding_cost_per_unit": 25,  # $ per unit per year
    "cost_per_km": 2.5,  # $ per km
    "vehicle_capacity": 1000,  # units per vehicle
    
    # Lead time
    "lead_time_days": 7,
    
    # Service level target
    "target_service_level": 0.95,
    
    # Warehouse location
    "warehouse": "warehouse"
}

# Create the problem statement for the workflow
problem_statement = f"""
Supply Chain Optimization Problem:

Company: {problem_data['company']}
Product: {problem_data['product']}
Delivery Regions: {', '.join(problem_data['regions'])}

Historical Monthly Demand (last 6 months): {problem_data['historical_demand']}
Current Inventory: {problem_data['current_inventory']} units

Cost Parameters:
- Ordering Cost: ${problem_data['ordering_cost']} per order
- Holding Cost: ${problem_data['holding_cost_per_unit']} per unit per year
- Transportation Cost: ${problem_data['cost_per_km']} per km

Objectives:
1. Forecast demand for the next 2 months
2. Determine optimal inventory levels
3. Plan delivery routes to {len(problem_data['regions'])} regions
4. Calculate total supply chain costs

Please provide analysis and recommendations.
"""

print(problem_statement)


Supply Chain Optimization Problem:

Company: Global Manufacturing Co.
Product: Industrial Components
Delivery Regions: region_A, region_B

Historical Monthly Demand (last 6 months): [850, 920, 950, 980, 1050, 1100]
Current Inventory: 1200 units

Cost Parameters:
- Ordering Cost: $500 per order
- Holding Cost: $25 per unit per year
- Transportation Cost: $2.5 per km

Objectives:
1. Forecast demand for the next 2 months
2. Determine optimal inventory levels
3. Plan delivery routes to 2 regions
4. Calculate total supply chain costs

Please provide analysis and recommendations.



## Step 6: Execute the Multi-Agent Workflow

Run the complete supply chain optimization with all four agents collaborating.

In [11]:
print("\n" + "=" * 70)
print("EXECUTING MULTI-AGENT SUPPLY CHAIN OPTIMIZATION WORKFLOW")
print("=" * 70)
print()
print("This will take a few minutes as each agent:")
print("  1. Receives input from previous agent")
print("  2. Calls its specialized MCP tools")
print("  3. Performs analysis with LLM reasoning")
print("  4. Passes results to next agent")
print()
print("Watch the agents collaborate in real-time!")
print("=" * 70)
print()

# Execute the workflow
result = supply_chain_workflow.run(problem_statement)

print("\n" + "=" * 70)
print("✓ WORKFLOW EXECUTION COMPLETE!")
print("=" * 70)


EXECUTING MULTI-AGENT SUPPLY CHAIN OPTIMIZATION WORKFLOW

This will take a few minutes as each agent:
  1. Receives input from previous agent
  2. Calls its specialized MCP tools
  3. Performs analysis with LLM reasoning
  4. Passes results to next agent

Watch the agents collaborate in real-time!

00:33:29 | INFO | Executing node: Demand_Forecasting


                    INFO     Executing node: Demand_Forecasting                                        ]8;id=794892;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py\logger.py]8;;\:]8;id=316518;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py#65\65]8;;\

00:33:29 🚀 agent_start 
00:33:29 | INFO | Agent started with task: 
Supply Chain Optimization Problem:

Company: Global Manufacturing Co.
Product: Industrial Components
Delivery Regions: region_A, region_B

Historical Monthly Demand (last 6 months): [850, 920, 950, 980, 1050, 1100]
Current Inventory: 1200 units

Cost Parameters:
- Ordering Cost: $500 per order
- Holding Cost: $25 per unit per year
- Transportation Cost: $2.5 per km

Objectives:
1. Forecast demand for the next 2 months
2. Determine optimal inventory levels
3. Plan delivery routes to 2 regions
4. Calculate total supply chain costs

Please provide analysis and recommendations.



                    INFO     Agent started with task:                                                  ]8;id=430288;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py\logger.py]8;;\:]8;id=975207;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py#65\65]8;;\
                             Supply Chain Optimization Problem:                                                    
                                                                                                                   
                             Company: Global Manufacturing Co.                                                     
                             Product: Industrial Components                                                        
                             Delivery Regions: region_A, region_B                                                  
                                                                                                                   
                             Historical Monthly Demand (last 6 months): [850, 920, 950, 980, 1050,                 
                             1100]                                                                                 
                             Current Inventory: 1200 units                                                         
                                                                                                                   
                             Cost Parameters:                                                                      
                             - Ordering Cost: $500 per order                                                       
                             - Holding Cost: $25 per unit per year                                                 
                             - Transportation Cost: $2.5 per km                                                    
                                                                                                                   
                             Objectives:                                                                           
                             1. Forecast demand for the next 2 months                                              
                             2. Determine optimal inventory levels                                                 
                             3. Plan delivery routes to 2 regions                                                  
                             4. Calculate total supply chain costs                                                 
                                                                                                                   
                             Please provide analysis and recommendations.                                          
                                                                                                                   

00:33:29 🔄 iteration_start [1]


...

00:38:23 🏁 agent_end 
00:38:23 | INFO | Agent finished. Success: True, Duration: 81.94s


                    INFO     Agent finished. Success: True, Duration: 81.94s                           ]8;id=9746;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py\logger.py]8;;\:]8;id=932694;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py#65\65]8;;\

00:38:23 | INFO | Workflow 'supply_chain_optimization' execution completed


                    INFO     Workflow 'supply_chain_optimization' execution completed                  ]8;id=218834;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py\logger.py]8;;\:]8;id=235576;file:///Users/aakashroy/Downloads/test/.venv/lib/python3.12/site-packages/or_af/utils/logger.py#65\65]8;;\


✓ WORKFLOW EXECUTION COMPLETE!


## Step 7: Analyze Results

Extract and display the recommendations from each agent.

In [13]:
print("\n" + "=" * 70)
print("SUPPLY CHAIN OPTIMIZATION RESULTS")
print("=" * 70)

# Get results from each node
node_results = result['results']
execution_order = result['execution_order']

# Map node IDs to names
node_names = {
    node_id: supply_chain_workflow.nodes[node_id].name 
    for node_id in execution_order
}

print(f"\nWorkflow: {result['workflow_name']}")
print(f"Status: {'SUCCESS ✓' if result['success'] else 'FAILED ✗'}")
print(f"Timestamp: {result['timestamp']}")
print(f"\nExecution Order: {' → '.join([node_names[nid] for nid in execution_order])}")

print("\n" + "=" * 70)

# Display results from each agent
for i, node_id in enumerate(execution_order, 1):
    node_name = node_names[node_id]
    node_result = node_results[node_id]
    
    print(f"\n{i}. {node_name}")
    print("-" * 70)
    print(f"Status: {node_result.status.value}")
    print(f"Execution Time: {node_result.execution_time:.4f}s")
    print(f"\nOutput:")
    
    # Try to parse output as JSON for better display
    try:
        if isinstance(node_result.output, str):
            # Check if it's an AgentResponse object string representation
            if 'response=' in node_result.output:
                # Extract the response part
                import re
                match = re.search(r"response='([^']*(?:'[^']*)*)'|response=\"([^\"]*)\"", node_result.output)
                if match:
                    response_text = match.group(1) or match.group(2)
                    print(response_text)
                else:
                    print(node_result.output)
            else:
                print(node_result.output)
        else:
            # If it's an AgentResponse object, get the response attribute
            if hasattr(node_result.output, 'response'):
                print(node_result.output.response)
            else:
                print(str(node_result.output))
    except Exception as e:
        print(node_result.output)
    
    print()

print("=" * 70)
print("\n✓ Multi-agent supply chain optimization completed successfully!")
print("\nKey Highlights:")
print("  • 4 specialized agents collaborated seamlessly")
print("  • 4 MCP servers provided 12+ specialized tools")
print("  • Each agent used LLM reasoning with domain-specific tools")
print("  • Workflow visualization showed clear agent interactions")
print("  • End-to-end OR problem solved with AI-powered optimization")


SUPPLY CHAIN OPTIMIZATION RESULTS

Workflow: supply_chain_optimization
Status: SUCCESS ✓
Timestamp: 2026-02-01 00:38:23.183675

Execution Order: Demand_Forecasting → Inventory_Optimization → Route_Planning → Cost_Analysis


1. Demand_Forecasting
----------------------------------------------------------------------
Status: completed
Execution Time: 100.7621s

Output:
{
  "forecast": {
    "method": "linear_regression_with_trend",
    "model_parameters": {
      "intercept": 808.0,
      "slope_per_month": 47.7143,
      "time_index_definition": "t=1..6 for the provided 6 months; forecasts are t=7,8"
    },
    "next_2_months": [
      {
        "period": "t+1",
        "point_forecast_units": 1142,
        "prediction_interval_95pct": {
          "lower": 1091,
          "upper": 1192
        }
      },
      {
        "period": "t+2",
        "point_forecast_units": 1190,
        "prediction_interval_95pct": {
          "lower": 1134,
          "upper": 1247
        }
      }
    ],


## Summary: Multi-Agent Architecture Benefits

This notebook demonstrated how OR-AF enables:

### 1. **Modular Design**
- Each agent specializes in one domain
- MCP servers provide isolated tool sets
- Easy to add/remove/modify agents

### 2. **Scalability**
- Add more agents for new capabilities
- Parallel execution possible (future feature)
- Each agent can be optimized independently

### 3. **Maintainability**
- Clear separation of concerns
- Each MCP server can be updated independently
- Workflow visualization aids debugging

### 4. **Real-World OR Applications**
- Classic OR models (EOQ, safety stock)
- Combined with modern AI reasoning
- Practical supply chain optimization

### 5. **OR-AF Framework Features Showcased**
- ✓ Multiple MCP Servers
- ✓ Multiple specialized agents
- ✓ Complex workflow graphs
- ✓ Workflow visualization (text, ASCII, Mermaid)
- ✓ Agent-to-agent data flow
- ✓ LLM reasoning with tool use
- ✓ Comprehensive logging and observability

---

**Next Steps:**
- Add parallel agent execution for independent tasks
- Implement conditional routing based on results
- Add A2A protocol for peer-to-peer agent communication
- Integrate real databases and APIs
- Add human-in-the-loop approval steps